# 2023IKCEST第五届“一带一路”国际大数据竞赛
# 一、背景介绍

本届大数据竞赛在中国工程院、教育部高等学校大学计算机课程教学指导委员会及丝绸之路大学联盟的指导下由联合国教科文组织国际工程科技知识中心（IKCEST）、中国工程科技知识中心（CKCEST）、百度公司及西安交通大学共同主办，旨在放眼“一带一路”倡议沿线国家，通过竞赛方式挖掘全球大数据人工智能尖端人才，实现政府—产业—高校合力推动大数据产业研究、应用、发展的目标，进一步夯实赛事的理论基础与实践基础，加快拔尖AI创新人才培养。

# 二、赛题介绍
随着新媒体时代信息媒介的多元化发展，各种内容大量活跃在媒体内中，与此同时各类虚假信息也充斥着社交媒体，影响着公众的判断和决策。如何在大量的文本、图像等多模态信息中，通过大数据与人工智能技术，纠正和消除虚假错误信息，对于网络舆情及社会治理有着重大意义。

本次赛题要求选手基于官方指定数据集，通过建模同一事实跨模态数据之间的关系 （主要是文本和图像），实现对任一模态信息能够进行虚假和真实性的检测。鼓励参赛选手通过大模型解决问题，进行技术探索。

In [4]:
#环境安装
# ! pip install paddlenlp==2.5.2

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


ERROR: Could not find a version that satisfies the requirement paddlenlp==2.5.2 (from versions: none)
ERROR: No matching distribution found for paddlenlp==2.5.2


In [1]:
import paddlenlp
print(dir(paddlenlp))

d:\Software\CodeApp\Anaconda\envs\paddle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Software\CodeApp\Anaconda\envs\paddle\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


['PADDLENLP_STABLE_VERSION', 'SimpleServer', 'Taskflow', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'data', 'dataaug', 'datasets', 'embeddings', 'experimental', 'layers', 'losses', 'metrics', 'ops', 'os', 'paddle', 'prompt', 'seq2vec', 'server', 'sys', 'taskflow', 'trainer', 'transformers', 'utils']


# 三、数据集介绍
本次比赛提供从国内外主流社交媒体平台上爬取的含有不同领域声明的数据集。

初赛：训练集与验证集： 提供中文训练集5694条以及英文数据4893条，同时公开英文验证集611条与中文验证集711条供选手优化模型。

初赛评测数据： 提供文娱、经济、健康领域的测试数据，这些领域的数据较容易区分。英文与中文数据集的测试集各600条。参赛队伍上传的结果文本的每一行就是对应的分类结果，该数据不公布，用于评测。


| 0 | 1 | 2 |
| -------- | -------- | -------- |
| non-rumor | rumor  | unverified |



[复赛数据后续见官网通知](https://aistudio.baidu.com/aistudio/competition/detail/1030/0/task-definition)

# 四、数据预处理
**数据集过大，右键选择解压/home/aistudio/data/data229919/data.zip数据集，耐心等待30分钟，直到出现以下文件夹和文件,解压之后硬盘达到约80g（压缩包27g、解压文件之后50g，可以将项目挂载的数据集取消，空余出27g）**
* test
* train
* val
* dataset_items_test.json
* dataset_items_train.json
* dataset_items_val.json

此处将数据集已经放置在queries_dataset_merge文件夹

In [1]:
from functools import partial
import numpy as np
import time
import os 
import copy
import json
import random
from tqdm import tqdm 

import paddle
from paddlenlp.datasets import load_dataset
import paddle.nn.functional as F
import paddle.nn as nn
import paddlenlp as ppnlp
from paddlenlp.transformers import LinearDecayWithWarmup
import pandas as pd

d:\Software\CodeApp\Anaconda\envs\paddle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Software\CodeApp\Anaconda\envs\paddle\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
#读取数据
import json
data_items_train = json.load(open("E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge/dataset_items_train.json"))
data_items_val = json.load(open("E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge/dataset_items_val.json"))
data_items_test = json.load(open("E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge/dataset_items_test.json"))

# 不用图片信息的，纯提取caption和qcap的

In [3]:
'''
读取数据中的每一个样本：图像img、文本caption、
对应的img_html_news、inverse_search为支持图像img和文本caption的证据材料
'''
import paddle
from paddle.vision import transforms as T
from paddle.io import Dataset
import json
from urllib.parse import urlparse
from PIL import Image
import os 
import imghdr

def process_string(input_str):
    input_str = input_str.replace('&#39;', '')
    input_str = input_str.replace('<b>','')
    input_str = input_str.replace('</b>','')
    #input_str = unidecode(input_str)  
    return input_str
    
class NewsContextDatasetEmbs(Dataset):
    # 初始化方法
    # 实例化对象的时候，会调用 __init__
    # self类的方法中第一个形参，指代当前的实例
    # 你实例化对象的时候，有对象的具体属性，但定义类的时候还没有，就用self来指代
    # 类内部使用self获取属性值
    def __init__(self, context_data_items_dict, queries_root_dir, split):
        self.context_data_items_dict = context_data_items_dict
        self.queries_root_dir = queries_root_dir
        self.idx_to_keys = list(context_data_items_dict.keys())
        self.transform =T.Compose([
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ])
        self.split=split
    def __len__(self):
        return len(self.context_data_items_dict)   

    # 从给定路径加载图像，并进行转RGB格式处理
    # def load_img_pil(self,image_path):
        # imghdr.what获取图片的类型
        if imghdr.what(image_path) == 'gif': 
            try:
                # 当时gif时，用image.open打开图像，因为他支持rgb、png、gif等。
                # 这里gif可能需要特殊处理下吧,不成功则要返回none
                with open(image_path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('RGB')
            except:
                return None 
        with open(image_path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    # 加载文件夹中的图像，将其转换为张量
    # def load_imgs_direct_search(self,item_folder_path,direct_dict):   # 字典，包含了需要处理的图像数据的信息
        list_imgs_tensors = [] # 空列表存储转换后的图像张量
        count = 0   # 记录处理的图像数量
        # 键列表
        keys_to_check = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        for key1 in keys_to_check:
            if key1 in direct_dict.keys():
                # 遍历该key1，对应的图像页面列表。从中提取图像路径
                for page in direct_dict[key1]:
                    image_path = os.path.join(item_folder_path,page['image_path'].split('/')[-1])
                    try:
                        pil_img = self.load_img_pil(image_path)
                    except Exception as e:
                        print(e)
                        print(image_path)
                    if pil_img == None: continue
                    # 按照init里面定义的处理下的图像，并ToTensor
                    transform_img = self.transform(pil_img)
                    count = count + 1 
                    list_imgs_tensors.append(transform_img)
        # 所有图像张量堆叠在第0维，并返回
        stacked_tensors = paddle.stack(list_imgs_tensors, axis=0)
        return stacked_tensors
    # 和上面逻辑类似，取caption对应的证据
    def load_captions(self,inv_dict):
        captions = ['']
        pages_with_captions_keys = ['all_fully_matched_captions','all_partially_matched_captions']
        for key1 in pages_with_captions_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    # 就拿caption中title和caption
                    if 'title' in page.keys():
                        item = page['title']
                        item = process_string(item)
                        captions.append(item)
                    
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
                    
        pages_with_title_only_keys = ['partially_matched_no_text','fully_matched_no_text']
        for key1 in pages_with_title_only_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    if 'title' in page.keys():
                        title = process_string(page['title'])
                        captions.append(title)
        return captions

    def load_captions_weibo(self,direct_dict):
        captions = ['']
        keys = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        for key1 in keys:
            if key1 in direct_dict.keys():
                for page in direct_dict[key1]:
                    if 'page_title' in page.keys():
                        item = page['page_title']
                        item = process_string(item)
                        captions.append(item)
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
        #print(captions)
        return captions
        #加载img文件夹
    def load_queries(self,key):
        caption = self.context_data_items_dict[key]['caption']
        # image_path = os.path.join(self.queries_root_dir,self.context_data_items_dict[key]['image_path'])
        # pil_img = self.load_img_pil(image_path)
        # transform_img = self.transform(pil_img)
        # return transform_img, caption
        return  caption
    # 魔法方法，用来重载类的索引操作符[]
    # 当使用instance[idx]的形式访问对象时，会自动调用该方法
    # enumerate会返回索引和对应的值的元组，使用值，也会调用这个方法
    # 返回sample。通过enumerate产生迭代器
    def __getitem__(self, idx):
        #print(idx)
        #print(self.context_data_items_dict)      
        #idx = idx.tolist()               
        key = self.idx_to_keys[idx]
        #print(key)
        item=self.context_data_items_dict.get(str(key))
        #print(item)
        # 如果为test没有label属性
        #print(self.split)
        if self.split=='train' or self.split=='val':
            label = int(item['label'])
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path'])
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path'])
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json')))
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            # imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)     
            # qImg,qCap =  self.load_queries(key)
            qCap =  self.load_queries(key)
            # sample = {'label': label, 'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
            sample = {'label': label, 'caption': captions,'qCap': qCap}
        else:
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path'])
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path'])
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json')))
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            # imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)     
            qCap =  self.load_queries(key)
            sample = {'caption': captions,'qCap': qCap}
        # print("================sample=======================")
        # print(sample)
        # print("================len(captions)=======================")
        # print(len(captions)) 
        # print("================type(imgs)=======================")
        # print(type(imgs))
        # print("================imgs.size=======================")
        # print(imgs.size)
        # print("================imgs.shape=======================")
        # print(imgs.shape) 
        # 所以batch的样子是 (sample,5,3) 
        # item[0]是sample
        # item[1]是5这样的数字
        # return sample,  len(captions), imgs.shape[0]
        return sample,len(captions)


# 原版的，包含图片的
读取数据中的每一个样本：图像img、文本caption、对应的img_html_news、inverse_search为支持图像img和文本caption的证据材料

In [6]:
import paddle
from paddle.vision import transforms as T
from paddle.io import Dataset
import json
from urllib.parse import urlparse
from PIL import Image
import os 
import imghdr
from paddleocr import PaddleOCR

def process_string(input_str):
    input_str = input_str.replace('&#39;', '')
    input_str = input_str.replace('<b>','')
    input_str = input_str.replace('</b>','')
    #input_str = unidecode(input_str)  
    return input_str
    
class NewsContextDatasetEmbs(Dataset):
    def __init__(self, context_data_items_dict, queries_root_dir, split):
        # json.load的json文件，里面每条是个词典
        self.context_data_items_dict = context_data_items_dict
        self.queries_root_dir = queries_root_dir
        # 获取json文件中的0、1、2~xxx
        self.idx_to_keys = list(context_data_items_dict.keys())
        self.transform =T.Compose([
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ])
        # 划分不同数据集执行不同步骤、train、val、test
        self.split=split
        self.ocr = PaddleOCR(use_angle_cls=True,lang="ch")
        # self.ocr1 = PaddleOCR(use_angle_cls=True,lang="ch")
    def __len__(self):
        # 数据集的长度
        return len(self.context_data_items_dict)   

    # 别的格式的图片全转为rgb
    def load_img_pil(self,image_path):
        # 根据图像文件的前几个字节识别图像文件格式。
        # 如果是gif转为rgb
        if imghdr.what(image_path) == 'gif': 
            try:
                with open(image_path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('RGB')
            except:
                return None 
        with open(image_path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    
    # 获取到数据集中用于证据的所有图片
    def load_imgs_direct_search(self,item_folder_path,direct_dict):   
        # list_imgs_tensors = []
        # count = 0   
        keys_to_check = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        img = ''
        for key1 in keys_to_check:
            if key1 in direct_dict.keys():
                for page in direct_dict[key1]:
                    image_path = os.path.join(item_folder_path,page['image_path'].split('/')[-1]) # 第二个参数拿到具体图片名称，eg：2433.jpg
                    # image_path = rf"{image_path}"
                    # print('type img_path'+"======"*10)
                    # print(type(image_path))
                    # print('image_path111111'+"======"*10)
                    # print(image_path)
                    # image_path = os.path.abspath(image_path)
                    # print('image_path-abspath'+"======"*10)
                    # print(image_path)
                    # image_path = str(image_path)
                    # image_path = image_path.replace("\\","/")
                    # print('image_path222'+"======"*10)
                    # print(image_path)
                    # image_path = rf"{image_path}"
                
                    # try:
                    #     pil_img = self.load_img_pil(image_path)
                    # except Exception as e:
                    #     print(e)
                    #     print(image_path)
                    # if pil_img == None: continue
                    if image_path == None: continue
                    # result =self.ocr1(image_path,cls=True)
                    result = self.ocr.ocr(image_path,cls=True)
                    result = result[0]
                    txts = [line[1][0] for line in result]
                    # img = ''
                    for t in txts:
                        img += t
                    # print('img111'+"======"*10)
                    # print(img)
                    # transform_img = self.transform(pil_img)
                    # count = count + 1 
                    # list_imgs_tensors.append(transform_img)
        # stacked_tensors = paddle.stack(list_imgs_tensors, axis=0)
        # print('img222'+"======"*10)
        # print(img)
        return img
    def load_captions(self,inv_dict):
        captions = ['']
        pages_with_captions_keys = ['all_fully_matched_captions','all_partially_matched_captions']
        for key1 in pages_with_captions_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    if 'title' in page.keys():
                        item = page['title']
                        item = process_string(item)
                        captions.append(item)
                    
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
                    
        pages_with_title_only_keys = ['partially_matched_no_text','fully_matched_no_text']
        for key1 in pages_with_title_only_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    if 'title' in page.keys():
                        title = process_string(page['title'])
                        captions.append(title)
        return captions

    def load_captions_weibo(self,direct_dict):
        captions = ['']
        keys = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        for key1 in keys:
            if key1 in direct_dict.keys():
                for page in direct_dict[key1]:
                    if 'page_title' in page.keys():
                        item = page['page_title']
                        item = process_string(item)
                        captions.append(item)
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
        #print(captions)
        return captions
    # 获取到img中的所有待证明图片
    def load_queries(self,key):
        caption = self.context_data_items_dict[key]['caption']
        image_path = os.path.join(self.queries_root_dir,self.context_data_items_dict[key]['image_path'])
        # pil_img = self.load_img_pil(image_path)
        # transform_img = self.transform(pil_img)
        result =self.ocr.ocr(image_path,cls=True)
        result = result[0]
        txts = [line[1][0] for line in result]
        qimg = ''
        for t in txts:
            qimg += t
        return qimg, caption
    def __getitem__(self, idx):
        #print(idx)
        #print(self.context_data_items_dict)      
        #idx = idx.tolist()          
        # key是当前对应第几个样本。test_dataset[953]中的953     
        key = self.idx_to_keys[idx]
        #print(key)
        # context_data_items_dict是那个总的json文件，根据key就是index索引到对应的内容
        item=self.context_data_items_dict.get(str(key))
        #print(item)
        # 如果为test没有label属性
        #print(self.split)
        if self.split=='train' or self.split=='val':
            label = paddle.to_tensor(int(item['label']))
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path']) # eg :.../Paddle2023IKCEST/queries_dataset_merge + val/img_html_news/389
            
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path']) # 索引到img_html_news和inverse具体样本的文件夹
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json'))) # 每个样本目录下的总json文件
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            # print('direct_dict'+"="*40)
            # print(direct_dict)
            # 传入的是样本目录，和它的json文件
            imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)     
            qImg,qCap =  self.load_queries(key)
            sample = {'label': label, 'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
        else:
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path'])
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path'])
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json')))
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)     
            qImg,qCap =  self.load_queries(key)
            sample = {'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
        
        
        #print(sample)
        #print(len(captions)) 
        #print(type(imgs))
        #print(imgs.size)
        #print(imgs.shape)  
        return sample


In [7]:
#### load Datasets ####
train_dataset = NewsContextDatasetEmbs(data_items_train, 'E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge','train')
val_dataset = NewsContextDatasetEmbs(data_items_val,'E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge','val')
test_dataset = NewsContextDatasetEmbs(data_items_test,'E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge','test')

[2023/09/14 13:58:56] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\lins/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\lins/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh

In [17]:
list = ['2asd', 'xcd', 'fff']
i = ''
b = 0
for a in list:
    b += 1
    a=f" {b}、"+a
    i += a
i


' 1、2asd 2、xcd 3、fff'

In [8]:
val_dataset[1]

[2023/09/14 13:59:16] ppocr DEBUG: dt_boxes num : 17, elapse : 2.0878753662109375
[2023/09/14 13:59:17] ppocr DEBUG: cls num  : 17, elapse : 0.030826807022094727
[2023/09/14 13:59:17] ppocr DEBUG: rec_res num  : 17, elapse : 0.1570885181427002
[2023/09/14 13:59:17] ppocr DEBUG: dt_boxes num : 13, elapse : 0.03324580192565918
[2023/09/14 13:59:17] ppocr DEBUG: cls num  : 13, elapse : 0.02223992347717285
[2023/09/14 13:59:17] ppocr DEBUG: rec_res num  : 13, elapse : 0.06400084495544434
[2023/09/14 13:59:17] ppocr DEBUG: dt_boxes num : 8, elapse : 0.0370326042175293
[2023/09/14 13:59:17] ppocr DEBUG: cls num  : 8, elapse : 0.016653776168823242
[2023/09/14 13:59:17] ppocr DEBUG: rec_res num  : 8, elapse : 0.03389096260070801
[2023/09/14 13:59:17] ppocr DEBUG: dt_boxes num : 0, elapse : 0.019058704376220703
[2023/09/14 13:59:17] ppocr DEBUG: cls num  : 0, elapse : 0
[2023/09/14 13:59:17] ppocr DEBUG: rec_res num  : 0, elapse : 0.0
[2023/09/14 13:59:17] ppocr DEBUG: dt_boxes num : 1, elapse 

{'label': Tensor(shape=[], dtype=int64, place=Place(gpu:0), stop_gradient=True,
        1),
 'caption': ['',
  '大理地震',
  '叙利亚义卖香皂遭遇抢购',
  '',
  '亚搏官方网站（中国）有限公司',
  '留学[超话]# 突发！！哈佛耶鲁法学院... 来自新浪留学- 微博',
  '突发[超话]#我来咯~新人报道，首发帖在线... 来自一声炮响- 微博',
  '焦点坊谈[超话]# 突发重磅消息：美国CNN消... 来自椒哥微记录- 微博'],
 'imgs': '-到此一游影视工作室V22-07-3011：21发布于北京来自张智霖超话致所有支持@張智囊先生的粉丝朋友们，很抱款地通知大家，原定于今日在杭州武林银泰百货举行的麦吉丽见面会活动，因张智需先生的身体原因，不得已临时取消。今日清晨，张智霖先生身体突发不适，随即送医急诊，经过医生的检查和诊断，确诊为胆囊炎。自前，张智霖先生正在医院接受治疗，所有的工作行程都将暂时延后。感谢大家的理解与支持，感谢麦吉丽品牌的理解，并配合处理活动的善后事宜，也请粉丝朋友们不要过度担优。突发情况给今天等候见面会的粉丝朋发们带来了困扰，工作室深感抱歉。期待尽快与大家更好的相见。??张智霖超话收藏3608白76Breaking News:Harvard andYalelawschoolsarewithdrawing from the U.S.News rankings, in perhaps thebiggest challengeyet to theschool ratings industry.TheNewYorkTimesYaleandHarvard LawSchoolsWithdrawFromtheU.S.NewsRankingsnytimes.com4:47PM.11/16/2S0cialFlow@西雅图夏至我关注了突发超话成为第17618位粉丝@一声炮响2022年09月17日18:24:32长按识别二维码加入微博超话，查看更多精彩帖子',
 'qImg': '@宋中垂建师',
 'qCap': '突发'}

In [9]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory
img_path = r'E:\Document\CodeSpace\Data_set\Paddle2023IKCEST\queries_dataset_merge/val\img_html_news\953\471.jpg'
# img_path = str(img_path)
# img_path = os.path.abspath(img_path)
print(type(img_path))
print(img_path)
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

print('done')
# # 显示结果
# from PIL import Image
# result = result[0]
# image = Image.open(img_path).convert('RGB')
# boxes = [line[0] for line in result]
# txts = [line[1][0] for line in result]
# scores = [line[1][1] for line in result]
# im_show = draw_ocr(image, boxes, txts, scores, font_path='lins.ttf')# 要出图，随便下载个字体
# im_show = Image.fromarray(im_show)
# im_show.save('result.jpg')

[2023/09/14 13:59:30] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\lins/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\lins/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh

# 制作添加了ocr出的文本的数据集

## train set ，ocr提取出保存为json

In [ ]:
import json
from tqdm import tqdm
# sample = {'label': label, 'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap}
# 假设 data 是一个包含100个数据的列表
data = val_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []
label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}

bar = tqdm(data)
for item in bar:
    # 提取 caption 和 qCap
    caption = item['caption']
    qCap = item['qCap']
    label = item['label']
    qImg = item['qImg']
    imgs = item['imgs']
    # input = 'qCap:'+str(qCap) + 'caption:'+str(caption)

    # 转换label为含义标签
    label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    output_data = {
        'label': label, 'caption': caption,'imgs': imgs,  'qImg': qImg, 'qCap': qCap
    }
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('train-ocr.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

## json再处理

### 把label字符串对应的倒数第三个数字提取出来
- `"label": "Tensor(shape=[1], dtype=int64, place=Place(gpu:0), stop_gradient=True,\n       [2])"` -> `2` ->unverified

In [26]:
import json


file = 'val-ocr.json'
label_mapping = {'0':'non-rumor','1':'rumor','2':'unverified'}

with open(file,encoding='utf-8') as f:
    data = json.load(f)

for item in data:
    label = item['label']   
    
    # 倒数第3个字符就是[]里的数字
    # number = label[-3]
    number = label
    
    # 替换label为提取的数字
    if label in label_mapping:
        temp = label_mapping[label]
    else:
        print(f'label {label} not defined in label_mapping')
    print("=================111")
    print(label_mapping[label])
    print("=================222")
    print(temp)
    item['label'] = temp

# 写回JSON文件    
with open(file, 'w',encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=================222
rumor
=================111
rumor
=

### 生成paddle数据集

In [27]:
from paddle.io import Dataset
from paddlenlp.datasets import MapDataset
import json
# data_items_train = json.load(open("E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge/dataset_items_train.json"))

class MyDataset(Dataset):
    def __init__(self, path):

        def load_data_from_source(path):
            data = json.load(open(path))
            print(data[0])
            return data

        self.data = load_data_from_source(path)

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)
data_path = "val-ocr.json"
ds = MyDataset(data_path)  # paddle.io.Dataset
new_ds = MapDataset(ds)    # paddlenlp.datasets.MapDataset

{'label': 'rumor', 'caption': ['', '多地网传“有人抢小孩”？真相是这样的…… - 新华网客户端', '2023年了！这些谣言你还会相信吗？中国互联网联合辟谣平台2022年度网络谣言盘点 - 新华网客户端', '2023年了！这些谣言你还会相信吗？中国互联网联合辟谣平台2022年度网络谣言盘点 - 新华网客户端', '多地网传“有人抢小孩”？真相是这样的……_中安在线', '2023年了！这些谣言你还会相信吗？_新华报业网', '', 'tyccl/cilin.txt at master · JoeWoo/tyccl · GitHub', 'chinese_correct_wsd/HIT-IRLab-同义词词林（扩展版）_full_2005.3.3.txt at master · taozhijiang/chinese_correct_wsd · GitHub', 'WordSimilarity/CoreSynonym.txt at master · TmengT/WordSimilarity · GitHub', 'Graduation-design/hit_synonym.txt at master · Baichenjia/Graduation-design · GitHub', 'Algorithms/cilin.txt at master · shijiebei2009/Algorithms · GitHub'], 'imgs': ' 1、JoeWoo/tyccl 2、tyccl（同义词词林）isarubygemthatprovides 3、friendlyfunctionstoanalysesimilaritybetween 4、ChineseWords. 5、81 6、47 7、15 8、Contributor 9、Issues 10、Stars 11、Forks 1、taozhijiang/ 2、chinese_correct_wsd 3、简易的中文纠错和消歧 4、81 5、264 6、134 7、Contributor 8、Issues 9、Stars 10、Forks 1、TmengT/ 2、WordSimilarity 3、中文文本词形，词序，词音，词性，词义多维相似度 4、计算 5、81 6、☆33 7、12

# 制作Qlora数据集代码

## 训练集

In [34]:
import json
import random

# 假设 data 是一个包含100个数据的列表
data = train_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
max_length = 15000

def count_words(string_list):
    total_words = 0
    for string in string_list:
        # 假设每个单词平均长度为5个字符
        average_word_length = 5
        words_in_string = len(string) // average_word_length
        total_words += words_in_string
    return total_words


for i,item in enumerate(data):
    total_length =0
    total_words =0
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    label = item[0]['label']
    total_length = sum(len(s) for s in caption)
    total_words= count_words(caption)
    # print("="*30)
    # print(caption)
    # print("="*30)
    # print('total_length:'+str(total_length)+"     total_words:"+str(total_words))
    if(total_length >max_length):
        print("="*30+"long-ling")
        caption = random.sample(caption,5)
        print(caption)
    input = "给定文本内容：qCap（待分类文本）:"+str(qCap) + 'caption（相关背景信息）:'+str(caption)+"请客观分析上下文信息,按以下三个类别给出qCap的分类:non-rumor: 可靠消息,无虚假或误导内容、rumor: 包含未经证实或错误信息的传闻、unverified: 真假难辨,无法判断属实与否。避免主观臆测或偏见。如果无法确定分类,选用unverified。"
    # 转换label为含义标签
    label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    # output_data = {
    #                    'instruction':input,
    #                    'output':label,
    #            }
    # output_data_list.append(output_data)
    output_data = input,
            
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('train-ocr.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

==============================long-ling
['国家卫健委：目前我国尚未发现不明原因儿童急性肝炎相关病例_接种_患儿 ...', "Biggest-Gaudiest-Patronuses I Spent Christmas Alone While My Family Got Together and Fought and Honestly I Feel So Good About This Biggest-Gaudiest-Patronuses I Bought Myself Presents Cooked a Big Meal Drank Wine and Watched Home Alone 3 Wl My Dog I Don't Feel Sorry for Myself at All Biggest-Gaudiest-Patronuses Meanwhile My Sister Threatened to Disown My Dad So He Threatened to Disown Her While My Mom Enabled and My Other Sister Downed a Bottle of Wine to Deal on the Other Side of the Country I Was Busy Discovering to My Deligh the That the 3rd Installment of the Home Alone Franchise Not Only Stood Up to the Test of Time but Contains Some of the Best Oneliners and Comedic Timing I Have Ever Seen Biggest-Gaudiest-Patronuses Honestly Keep Toxic People Out of Your Life Family Isn't Everything and Bad Family Is Often Worse Than No Family Don't Be Afraid to Put in Some Distance Especially During High Stress 

## val

In [30]:
import json
import random

# 假设 data 是一个包含100个数据的列表
data = val_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
max_length = 15000

def count_words(string_list):
    total_words = 0
    for string in string_list:
        # 假设每个单词平均长度为5个字符
        average_word_length = 5
        words_in_string = len(string) // average_word_length
        total_words += words_in_string
    return total_words


for i,item in enumerate(data):
    total_length =0
    total_words =0
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    label = item[0]['label']
    total_length = sum(len(s) for s in caption)
    total_words= count_words(caption)
    # print("="*30)
    # print(caption)
    # print("="*30)
    # print('total_length:'+str(total_length)+"     total_words:"+str(total_words))
    if(total_length >max_length):
        print("="*30+"long-ling")
        caption = random.sample(caption,5)
        print(caption)
    input = "给定文本内容：qCap（待分类文本）:"+str(qCap) + 'caption（相关背景信息）:'+str(caption)+"请客观分析上下文信息,按以下三个类别给出qCap的分类:non-rumor: 可靠消息,无虚假或误导内容、rumor: 包含未经证实或错误信息的传闻、unverified: 真假难辨,无法判断属实与否。避免主观臆测或偏见。如果无法确定分类,选用unverified。"
    # 转换label为含义标签
    label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    output_data = {
                       'instruction':input,
                       'output':label,
               }
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('val.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

# 制作prompt tuning数据集代码

### train训练集

In [18]:
import json

# 假设 data 是一个包含100个数据的列表
data = train_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
for item in data:
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    label = item[0]['label']
    input = 'qCap:'+str(qCap) + 'caption:'+str(caption)

    # 转换label为含义标签
    label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    output_data = {'instruction':'''Based on the content in the caption below, determine whether qCap is a rumor. Only select the one with the highest probability among non-rumor, rumor, and unverified as the output.''',
                   'output':label,
                   'input':input}
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('train1.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

### val 验证集

In [11]:
import json

# 假设 data 是一个包含100个数据的列表
data = val_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
for item in data:
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    label = item[0]['label']

    # 转换label为含义标签
    label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    output_data = {'label':label,'context':{'instruction':'''Based on the content in the caption in the following content, to determine whether qCap is a rumor, the thinking steps are as follows: 1. Find out the content related to qCap from the caption. 2. Determine whether it is a rumor based on the relevant content found in step 1. 3. Select only the one that you think has the highest probability among non-rumor, rumor, and unverified as the output.''',
'qCap': qCap,
'caption': caption,
}}
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('val.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

### test 测试集 推理集

In [19]:
import json

# 假设 data 是一个包含100个数据的列表
data = test_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
for item in data:
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    # label = item[0]['label']
    input = 'qCap:'+str(qCap) + 'caption:'+str(caption)

    # 转换label为含义标签
    # label = label_mapping.get(label,str(label))

    # 将数据保存为字典
    output_data = {
        'instruction':'''Based on the content in the caption below, determine whether qCap is a rumor. Only select the one with the highest probability among non-rumor, rumor, and unverified as the output.''',
        'input':input}
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

### 抽取点组成个小的数据集

In [9]:
import json
import random

# 读取整个 JSON 文件
with open('train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 计算每个部分应该抽取的数量
total_samples = 200
num_parts = 5  # 假设分成五个部分
samples_per_part = total_samples // num_parts

# 对数据进行随机打乱
random.shuffle(data)

# 初始化抽样结果列表
sampled_data = []

# 从每个部分分别抽取相应数量的数据
for i in range(num_parts):
    start_index = i * samples_per_part
    end_index = (i + 1) * samples_per_part
    sampled_data.extend(data[start_index:end_index])

# 将抽样的数据保存为新的 JSON 文件
with open('sampled_output_train_200.json', 'w', encoding='utf-8') as f:
    json.dump(sampled_data, f, ensure_ascii=False)

# 制作0.86方案数据集

## 训练集

In [33]:
import json
import random

# 假设 data 是一个包含100个数据的列表
data = train_dataset

# 初始化一个空列表，用于存储每个数据的 caption 和 qCap  和 label字段
output_data_list = []

label_mapping = {0:'non-rumor',1:'rumor',2:'unverified'}
max_length = 15000

def count_words(string_list):
    total_words = 0
    for string in string_list:
        # 假设每个单词平均长度为5个字符
        average_word_length = 5
        words_in_string = len(string) // average_word_length
        total_words += words_in_string
    return total_words


for i,item in enumerate(data):
    total_length =0
    total_words =0
    # 提取 caption 和 qCap
    caption = item[0]['caption']
    qCap = item[0]['qCap']
    label = item[0]['label']
     # 转换label为含义标签
    label = label_mapping.get(label,str(label))
    # total_length = sum(len(s) for s in caption)
    # total_words= count_words(caption)
    # print("="*30)
    # print(caption)
    # print("="*30)
    # print('total_length:'+str(total_length)+"     total_words:"+str(total_words))
    # if(total_length >max_length):
    #     print("="*30+"long-ling")
    #     caption = random.sample(caption,5)
    #     print(caption)
    # input = "给定文本内容：qCap（待分类文本）:"+str(qCap) + 'caption（相关背景信息）:'+str(caption)+"请客观分析上下文信息,按以下三个类别给出qCap的分类:non-rumor: 可靠消息,无虚假或误导内容、rumor: 包含未经证实或错误信息的传闻、unverified: 真假难辨,无法判断属实与否。避免主观臆测或偏见。如果无法确定分类,选用unverified。"
   

    # 将数据保存为字典
    output_data = [label]
    output_data_list.append(output_data)

# 将列表保存为 JSON 文件
with open('train.json', 'w', encoding='utf-8') as f:
    json.dump(output_data_list, f, ensure_ascii=False)

In [16]:
train_dataset[1]

================sample=======================
{'label': Tensor(shape=[], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       1), 'caption': ['', '御书房 知识分享站 - 骂醒自己的6句话\n. \n. \n. \n. \n. \n. \n. \n[ 荣誉赞助 : WINBOX... | Facebook', 'Sina Visitor System', '', '中国金茂前3个月销售额442.3亿元，此前否认并入华润置地传闻|中国金茂_新浪财经_新浪网', '金茂要与华润置业合并？金茂出口否认_腾讯新闻', 'һ\\xcc챩\\xb5\\xf840%\\xa3\\xa1\\xd3\\xd6һ\\xb7\\xbf\\xc6\U000e1c33Ų\\xbbס\\xc1ˡ\\xb1\\xa3\\xbf - \\xcd\\xf2ά\\xb6\\xc1\\xd5\\xdf\\xcd\\xf8', '详情页', '安永高管被举报违规买入华润置地房产_新浪网'], 'qCap': '网传中国金茂并入华润置地 '}
================len(captions)=======================
9
================type(imgs)=======================
<class 'paddle.Tensor'>
================imgs.size=======================
752640
================imgs.shape=======================
[5, 3, 224, 224]


({'label': Tensor(shape=[], dtype=int64, place=Place(gpu:0), stop_gradient=True,
         1),
  'caption': ['',
   '御书房 知识分享站 - 骂醒自己的6句话\n. \n. \n. \n. \n. \n. \n. \n[ 荣誉赞助 : WINBOX... | Facebook',
   'Sina Visitor System',
   '',
   '中国金茂前3个月销售额442.3亿元，此前否认并入华润置地传闻|中国金茂_新浪财经_新浪网',
   '金茂要与华润置业合并？金茂出口否认_腾讯新闻',
   'һ\\xcc챩\\xb5\\xf840%\\xa3\\xa1\\xd3\\xd6һ\\xb7\\xbf\\xc6\U000e1c33Ų\\xbbס\\xc1ˡ\\xb1\\xa3\\xbf - \\xcd\\xf2ά\\xb6\\xc1\\xd5\\xdf\\xcd\\xf8',
   '详情页',
   '安永高管被举报违规买入华润置地房产_新浪网'],
  'qCap': '网传中国金茂并入华润置地 '},
 9,
 5)

In [11]:
# 打印数据
# for step, batch in enumerate(train_dataset, start=1):
#     print(batch)
#     break

================sample=======================
{'label': Tensor(shape=[], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       1), 'caption': ['', '上海浦东近日发生强奸杀人案？谣言！ - 观察者', '点击查看大图', '上海浦东近日发生强奸杀人案？谣言！ - guancha.cn', '点击查看大图', 'Attention Required! | Cloudflare', '上海浦东近日发生强奸杀人案？谣言！杜撰者已被警方批评教育', '上海浦东近日发生强奸杀人案？谣言！-观察者网', '', '乐平奸杀案嫌犯获不起诉决定  律师:来得太晚了|检察院|嫌犯|奸杀案_新浪新闻', '景德镇检察院做出不起诉决定。', '上海浦东供电公司2020录用情况 - 知乎', '上海香港旧金山，去美国攻略 - 知乎', '谷歌上海面试经验 - 知乎', '怎么用 Apple Watch 打电话？看这篇就够 - 知乎'], 'imgs': Tensor(shape=[5, 3, 224, 224], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[[[ 1.42692029,  1.58104312,  1.58104312, ...,  1.92353821,
            1.97491241,  1.97491241],
          [ 1.46116984,  1.58104312,  1.58104312, ...,  1.92353821,
            1.97491241,  1.99203718],
          [ 1.47829461,  1.56391835,  1.58104312, ...,  1.92353821,
            1.95778763,  1.99203718],
          ...,
          [ 1.46116984,  1.42692029,  1.40979564, ...,  0.77617949,
            0

###  collate函数通常作为DataLoader函数的参数,用来处理数据得到数据集

In [ ]:
import paddle 
# collate函数通常与DataLoader（数据加载器）关联
# 接受一个列表作为输入，该列表包含多个样本，然后根据指定的规则将这些样本组合成一个批次
# 作为DataLoader函数的参数
# 处理批量训练数据 batch，返回labels, cap_batch, img_batch, qCap_batch, qImg_batch
def collate_context_bert_train(batch):
    #print(batch)
    # item[0]是sample，所有sample存里面
    samples = [item[0] for item in batch]
    # 将caption中最大值的得到。
    max_captions_len = max([item[1] for item in batch])
    max_images_len = max([item[2] for item in batch])
    qCap_batch = []
    qImg_batch = []
    img_batch = []
    cap_batch = []
    labels = [] 
    for j in range(0,len(samples)):  
        sample = samples[j]    
        # 是字典形式。所以sample['label']得到值
        labels.append(sample['label'])
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(0,max_captions_len-cap_len):
            captions.append("")
        if len(sample['imgs'].shape) > 2:
            padding_size = (max_images_len-sample['imgs'].shape[0], sample['imgs'].shape[1], sample['imgs'].shape[2], sample['imgs'].shape[3])
        else:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1])
        padded_mem_img = paddle.concat((sample['imgs'], paddle.zeros(padding_size)),axis=0)
        #print(1)
        img_batch.append(padded_mem_img)#pad证据图片
        cap_batch.append(captions)
        qImg_batch.append(sample['qImg'])#[3, 224, 224]
        qCap_batch.append(sample['qCap'])     
    #print(labels)   
    #print(img_batch)
    img_batch = paddle.stack(img_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    labels = paddle.stack(labels, axis=0) 
    #print(3)  
    return labels, cap_batch, img_batch, qCap_batch, qImg_batch

def collate_context_bert_test(batch):
    samples = [item[0] for item in batch]
    max_captions_len = max([item[1] for item in batch])
    max_images_len = max([item[2] for item in batch])
    qCap_batch = []
    qImg_batch = []
    img_batch = []
    cap_batch = []
    for j in range(0,len(samples)):  
        sample = samples[j]    
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(0,max_captions_len-cap_len):
            captions.append("")
        if len(sample['imgs'].shape) > 2:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1],sample['imgs'].shape[2],sample['imgs'].shape[3])
        else:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1])
        padded_mem_img = paddle.concat((sample['imgs'], paddle.zeros(padding_size)),axis=0)
        img_batch.append(padded_mem_img)
        cap_batch.append(captions)
        qImg_batch.append(sample['qImg'])
        qCap_batch.append(sample['qCap'])        
    img_batch = paddle.stack(img_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    return cap_batch, img_batch, qCap_batch, qImg_batch

In [ ]:
# load DataLoader
from paddle.io import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn = collate_context_bert_train, return_list=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn = collate_context_bert_train,  return_list=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn = collate_context_bert_test, return_list=True)

In [ ]:
# 打印数据
for step, batch in enumerate(train_dataloader, start=1):
    print(batch)
    input_test = batch
    break

# 四、模型构建
**本次赛题为一个NLP与多模态的分类赛题，整体建模采用特征提取、特征交互、预测分类三个阶段**

**特征提取：** 对于图像数据，使用ResNet模型进行特征提取、对于文本数据，使用预训练模型Ernie-m多语言模型对中文和英文同时处理，qCap,qImg,（需要验证的标题或图像材料）、caps,imgs（支持验证的文本、图像证据材料）

**特征交互**：使用多头自注意力机制，将标题与文本证据材料交互、图像与图像证据材料交互，输出与需要验证的标题和图像的相关证据特征caps_feature、imgs_features

**预测分类：** 最后使用全连接层将标题特征、图像特征、相关的文本证据特征、相关的图像证据特征拼接输入到分类器得到最终结果
![](https://ai-studio-static-online.cdn.bcebos.com/3f29e3f853b9445fbeb24189103cdbbcb8364498dc484593a891839994dadbd6)



## 多语言预训练模型ERNIE-M
2021年，百度发布多语言预训练模型ERNIE-M。ERNIE-M通过对96门语言的学习，使得一个模型能同时理解96种语言，该项技术在5类典型跨语言理解任务上刷新世界最好效果。

## ERNIE-M原理
ERNIE-M基于飞桨PaddlePaddle框架训练，该模型构建了大小为25万的多语言词表，涵盖了96种语言的大多数常见词汇，训练语料包含了汉语、英语、法语、南非语、阿尔巴尼亚语、阿姆哈拉语、梵语、阿拉伯语、亚美尼亚语、阿萨姆语、阿塞拜疆语等96种语言，约1.5万亿字符。

ERNIE-M的学习过程由两阶段组成。第一阶段从少量的双语语料中学习跨语言理解能力，使模型学到初步的语言对齐关系；第二阶段使用回译的思想，通过大量的单语语料学习，增强模型的跨语言理解能力。

[百度NLP知乎介绍](https://zhuanlan.zhihu.com/p/344810337)


In [ ]:
from paddle.vision import models
import paddle
from paddlenlp.transformers import ErnieMModel,ErnieMTokenizer
from paddle.nn import functional as F
from paddle import nn
import matplotlib.pyplot as plt
import numpy as np

from visualdl import LogWriter

class EncoderCNN(nn.Layer):
    def __init__(self, resnet_arch = 'resnet101'):
        super(EncoderCNN, self).__init__()
        if resnet_arch == 'resnet101':
            resnet = models.resnet101(pretrained=True)
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)
        self.adaptive_pool = nn.AdaptiveAvgPool2D((1, 1))
    def forward(self, images, features='pool'):
        out = self.resnet(images)
        if features == 'pool':
            out = self.adaptive_pool(out)
            out = paddle.reshape(out, (out.shape[0],out.shape[1]))
        return out

class NetWork(nn.Layer):
    def __init__(self, mode):
        super(NetWork, self).__init__()
        self.mode = mode           
        self.ernie = ErnieMModel.from_pretrained('THUDM/chatglm2-6b')
        self.tokenizer = ErnieMTokenizer.from_pretrained('THUDM/chatglm2-6b')
        self.resnet = EncoderCNN()
        self.classifier1 = nn.Linear(2*(768+2048),1024)
        self.classifier2 = nn.Linear(1024,3)
        self.attention_text = nn.MultiHeadAttention(768,16)
        self.attention_image = nn.MultiHeadAttention(2048,16)
        if self.mode == 'text':
            self.classifier = nn.Linear(768,3)
        self.resnet.eval()

    def forward(self,qCap,qImg,caps,imgs):
        self.resnet.eval()
        encode_dict_qcap = self.tokenizer(text = qCap ,max_length = 128 ,truncation=True, padding='max_length')
        input_ids_qcap = encode_dict_qcap['input_ids']
        input_ids_qcap = paddle.to_tensor(input_ids_qcap)
        qcap_feature, pooled_output= self.ernie(input_ids_qcap) #(b,length,dim)
        if self.mode == 'text':
            logits = self.classifier(qcap_feature[:,0,:].squeeze(1))
            return logits
        caps_feature = []
        for i,caption in enumerate (caps):
            encode_dict_cap = self.tokenizer(text = caption ,max_length = 128 ,truncation=True, padding='max_length')
            input_ids_caps = encode_dict_cap['input_ids']
            input_ids_caps = paddle.to_tensor(input_ids_caps)
            cap_feature, pooled_output= self.ernie(input_ids_caps) #(b,length,dim)
            caps_feature.append(cap_feature)
        caps_feature = paddle.stack(caps_feature,axis=0) #(b,num,length,dim)
        caps_feature = caps_feature.mean(axis=1)#(b,length,dim)
        caps_feature = self.attention_text(qcap_feature,caps_feature,caps_feature) #(b,length,dim)
        imgs_features = []
        for img in imgs:
            imgs_feature = self.resnet(img) #(length,dim)
            imgs_features.append(imgs_feature)
        imgs_features = paddle.stack(imgs_features,axis=0) #(b,length,dim)
        qImg_features = []
        for qImage in qImg:
            qImg_feature = self.resnet(qImage.unsqueeze(axis=0)) #(1,dim)
            qImg_features.append(qImg_feature)
        qImg_feature = paddle.stack(qImg_features,axis=0) #(b,1,dim)
        imgs_features = self.attention_image(qImg_feature,imgs_features,imgs_features) #(b,1,dim)
        # [1, 128, 768] [1, 128, 768] [1, 1, 2048] [1, 1, 2048] origin
        # print(qcap_feature.shape,caps_feature.shape,qImg_feature.shape,imgs_features.shape)
        # print((qcap_feature[:,0,:].shape,caps_feature[:,0,:].shape,qImg_feature.squeeze(1).shape,imgs_features.squeeze(1).shape))
        # ([1,768], [1 , 768], [1, 2048], [1,  2048])
        feature = paddle.concat(x=[qcap_feature[:,0,:], caps_feature[:,0,:], qImg_feature.squeeze(1), imgs_features.squeeze(1)], axis=-1) 
        logits = self.classifier1(feature)
        logits = self.classifier2(logits)
        return logits

In [ ]:
# 声明模型
model = NetWork("image")
print(model)



[2023-09-05 16:56:51,843] [    INFO] - Downloading config.json from https://bj.bcebos.com/paddlenlp/models/community/THUDM/chatglm2-6b/config.json
100%|██████████| 885/885 [00:00<?, ?B/s] 
[2023-09-05 16:56:52,081] [    INFO] - loading configuration file C:\Users\lins\.paddlenlp\models\THUDM/chatglm2-6b\config.json
[2023-09-05 16:56:52,082] [ WARNING] - You are using a model of type chatglm_v2 to instantiate a model of type ernie_m. This is not supported for all configurations of models and can yield errors.
[2023-09-05 16:56:52,084] [    INFO] - Model config ErnieMConfig {
  "_name_or_path": "THUDM/chatglm2-6b",
  "add_bias_linear": false,
  "add_qkv_bias": true,
  "apply_query_key_layer_scaling": true,
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "ChatGLMv2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.1,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "dtype": "float16",
  "eos_token_id": 2,
  

KeyboardInterrupt: 

# 六、训练配置

In [ ]:
epochs = 2
num_training_steps = len(train_dataloader) * epochs
warmup_steps = int(num_training_steps*0.1)
print(num_training_steps,warmup_steps)
# 定义 learning_rate_scheduler，负责在训练过程中对 lr 进行调度
lr_scheduler = LinearDecayWithWarmup(1e-6, num_training_steps, warmup_steps)
# 训练结束后，存储模型参数
save_dir ="checkpoint/"
best_dir = "best_model"
# 创建保存的文件夹
os.makedirs(save_dir,exist_ok=True)
os.makedirs(best_dir,exist_ok=True)

decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义 Optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=1.2e-4,
    apply_decay_param_fun=lambda x: x in decay_params)

# 交叉熵损失
criterion = paddle.nn.loss.CrossEntropyLoss()

# 评估的时候采用准确率指标
metric = paddle.metric.Accuracy()

5592 559


# 七、模型训练

In [ ]:
# 定义线下评估 评价指标为acc 线上评估是f1score
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:      
        labels, cap_batch, img_batch, qCap_batch, qImg_batch = batch
        logits = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()
    return np.mean(losses), accu

In [ ]:
# 定义训练
def do_train(model, criterion, metric, val_dataloader,train_dataloader):
    print("train run start")
    global_step = 0
    tic_train = time.time()
    best_accuracy=0.0
    for epoch in range(1, epochs + 1):
        for step, batch in enumerate(train_dataloader, start=1):
            labels, cap_batch, img_batch, qCap_batch, qImg_batch = batch
            probs = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch)
            
            # 16GB显存崩了。
            # # 用visualDL，绘制模型图
            # net=model()
            # with LogWriter(logdir="./log/graph_test/") as writer:
            #     writer.add_graph(
            #         model=net,
            #         input_spec=[qCap_batch,qImg_batch,cap_batch,img_batch],
            #         verbose=True)
            # break
            loss = criterion(probs, labels)
            correct = metric.compute(probs, labels)
            metric.update(correct)
            acc = metric.accumulate()

            global_step += 1 
            # 每间隔 100 step 输出训练指标
            if global_step % 1 == 0:
                print(
                    "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, step, loss, acc,
                        10 / (time.time() - tic_train)))
                tic_train = time.time()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()

            # 每间隔一个epoch 在验证集进行评估
            if global_step % len(train_dataloader) == 0:
                eval_loss,eval_accu=evaluate(model, criterion, metric, val_dataloader)
                save_param_path = os.path.join(save_dir+str(epoch), 'model_state.pdparams')
                paddle.save(model.state_dict(), save_param_path)
                if(best_accuracy<eval_accu):
                    best_accuracy=eval_accu
                    # 保存模型
                    save_param_path = os.path.join(best_dir, 'model_best.pdparams')
                    paddle.save(model.state_dict(), save_param_path)
do_train(model, criterion, metric, val_dataloader,train_dataloader) 

train run start
image file is truncated (8 bytes not processed)
E:/Document/CodeSpace/Data_set/Paddle2023IKCEST/queries_dataset_merge\train/img_html_news/37\6068.jpg
global step 1, epoch: 1, batch: 1, loss: 1.15941, accu: 0.50000, speed: 1.69 step/s
global step 2, epoch: 1, batch: 2, loss: 1.76783, accu: 0.37500, speed: 2.03 step/s
global step 3, epoch: 1, batch: 3, loss: 1.59816, accu: 0.33333, speed: 0.60 step/s
global step 4, epoch: 1, batch: 4, loss: 2.71103, accu: 0.25000, speed: 0.24 step/s
global step 5, epoch: 1, batch: 5, loss: 1.57983, accu: 0.30000, speed: 0.38 step/s
global step 6, epoch: 1, batch: 6, loss: 1.86186, accu: 0.25000, speed: 0.43 step/s
global step 7, epoch: 1, batch: 7, loss: 1.88637, accu: 0.21429, speed: 0.40 step/s
global step 8, epoch: 1, batch: 8, loss: 2.36982, accu: 0.21875, speed: 0.32 step/s
global step 9, epoch: 1, batch: 9, loss: 2.23455, accu: 0.19444, speed: 0.24 step/s
global step 10, epoch: 1, batch: 10, loss: 1.93948, accu: 0.20000, speed: 0.28

KeyboardInterrupt: 

# 八、模型预测
**模型预测前，请重启内核，清空占用的显存**

In [ ]:
# 根据实际运行情况，更换加载的参数路径
import os
import paddle

params_path = 'checkpoint/model_best.pdparams'
if params_path and os.path.isfile(params_path):
    # 加载模型参数
    state_dict = paddle.load(params_path)
    model.set_dict(state_dict)
    print("Loaded parameters from %s" % params_path)

In [ ]:
results = []
# 切换model模型为评估模式，关闭dropout等随机因素
model.eval()
count=0
for batch in test_dataloader:
    count+=1
    cap_batch, img_batch, qCap_batch, qImg_batch = batch
    logits = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch)
    
                
    # 报错AttributeError: 'Tensor' object has no attribute '_full_name'，先不管了
    # 用visualDL，绘制模型图
    # net=model(qCap_batch,qImg_batch,cap_batch,img_batch)
    # with LogWriter(logdir="./log/graph_test/") as writer:
    #     writer.add_graph(
    #         model=net,
    #         input_spec=[qCap_batch,qImg_batch,cap_batch,img_batch],
    #         verbose=True)
    # break
    
    # 预测分类
    probs = F.softmax(logits, axis=-1)
    label = paddle.argmax(probs, axis=1).numpy()
    results += label.tolist()
    print(count)
print(results[:5])
print(len(results))

Failed to save model graph, error: 'Tensor' object has no attribute '_full_name'


AttributeError: 'Tensor' object has no attribute '_full_name'

In [ ]:
# 输出结果
import pandas as pd
#id/label
#字典中的key值即为csv中的列名
id_list=range(len(results))
print(id_list)
frame = pd.DataFrame({'id':id_list,'label':results})
frame.to_csv("result.csv",index=False,sep=',')

range(0, 50)


# 九、后续优化

baseline分数只有65分，还有很大的改进地方，大家多多尝试，下面是一些想法

参数调优：学习率、优化器以及其他超参数等

特征提取：更换预训练权重更大的图像特征提取器or文本特征提取器（Ernie or Bert系列）

特征交互：目前使用多头自注意力机制对文本与文本证据交互、图像与图像证据交互，可以尝试文本与图像之间的跨模态交互
